# ViewSet视图集
- ViewSet视图集，相当于是之前所提到的视图的一个集合，在视图集中，不再有get和post，取而代之的是list和create

## 基本使用
- 若想实现一个包含增，删，改，查，列表的视图集，可以通过以下代码来实现

        from rest_framework import viewsets
        from .models import Merchant
        from .serializers import MerchantSerializer
        from rest_framework.response import Response
        from rest_framework import status
        from django.shortcuts import get_object_or_404

        class MerchantViewSet(viewsets.ViewSet):
            def list(self,request):
                queryset = Merchant.objects.all()
                serializer = MerchantSerializer(queryset,many=True)
                return Response(data=serializer.data)

            def create(self,request):
                serializer = MerchantSerializer(data=request.data)
                if serializer.is_valid():
                    serializer.save()
                    return Response("success")
                else:
                    return Response("fail",status=status.HTTP_400_BAD_REQUEST)

            def retrieve(self,request,pk=None):
                queryset = Merchant.objects.all()
                merchant = get_object_or_404(queryset,pk=pk)
                serializer = MerchantSerializer(merchant)
                return Response(serializer.data)

            def update(self,request,pk=None):
                queryset = Merchant.objects.all()
                merchant = get_object_or_404(queryset, pk=pk)
                serializer = MerchantSerializer()
                serializer.update(merchant,request.data)
                return Response('success')

            def destroy(self,request,pk=None):
                queryset = Merchant.objects.all()
                merchant = get_object_or_404(queryset,pk=pk)
                merchant.delete()
                return Response('success')
                
- 然后在urls.py中，通过rest_framework.routers.DefaultRouter注册路由即可
        router = DefaultRouter()
        router.register("merchant",MerchantViewSet,basename="merchant")

        urlpatterns = []+router.urls
        

# ModelViewSet
- 因为我们一个视图基本上都是针对一个模型进行操作的，那么增删改查操作针对的也就是不同的模型，所以可以使用ModelViewSet简化以上的代码
        class MerchantViewSet(viewsets.ModelViewSet):
        queryset = Merchant.objects.all()
        serializer_class = MerchantSerializer
        
- 有时候在一个视图中，还需要增加其他的url，这时候就可以使用@action来实现
        from rest_framework.decorators import action

        from rest_framework.decorators import action

        class MerchantViewSet(viewsets.ModelViewSet):
            queryset = Merchant.objects.all()
            serializer_class = MerchantSerializer

            @action(['GET'],detail=False)
            def cs(self,request,*args,**kwargs):
                queryset = self.get_queryset()
                queryset = queryset.filter(name__contains="长沙")
                serializer = MerchantSerializer(queryset,many=True)
                return Response(serializer.data)
- urls.py路由部分不需要修改，以后可以直接通过/merchant/cs/可以访问到name中包含了“长沙”两个字的所有商家